## Task 2

### Task 2.1
Consists of extracting data from the dictionary and adding the same in a nested dictionary & storing them as JSON files

In [ ]:
from bs4 import BeautifulSoup
import os 
import re
import nltk
from copy import deepcopy
import json

In [ ]:
ECTNestedDict = {}
transcript = {}
file_path = os.getcwd() + "/data/"

script_file_path = os.getcwd() + "/ECTNestedDict/"
os.mkdir(script_file_path)
file_list = os.listdir(file_path)
debug_list = []

# Dictionary Construction
for i in range(len(file_list)):
    path = file_path + file_list[i]
    ID = re.findall(r'\d+', file_list[i])[0]
    # print("DocID: " + ID)

    tr_file = open(path)
    # Parsing the html
    soup = BeautifulSoup(tr_file, features="html.parser")

    # kill all script and style elements
    for script in soup(["script", "style"]):
        script.extract()    # rip it out

    #  --------------------------------------------------------------------------------------------------------------
    # Finding & Storing Date
    # Storing the 1st line
    a = (soup.find('p', {'class' : 'p p1'})).get_text()  # For finding a specific class in the html
    #  Finding the date using regex
    a = re.findall(r'[A-Z]\w+\s+\d+\,\s+\d+',a)

    
    # Brute-Forcing the "p p_1" class in case Dat not in heading
    if(not a): 
        temp = soup.findAll('p', class_="p p1")
        for j in range(5): # Checking in the 1st 5 tags
            a = re.findall(r'[A-Z]\w+\s+\d+\,\s+\d+', temp[j].get_text())
            if(not a):
                a = re.findall(r'[A-Z]\w+\s+\d+\s+\d+', temp[j].get_text())
            if(a):
                break
    if(not a):
        print(temp[j].get_text())
    # Removing extra spaces from the date
    date = " ".join(a[0].split())

    #  --------------------------------------------------------------------------------------------------------------
    # Constructing Participant List
    a = soup.findAll('p')
    part_list= []
    flag=0  # flag for storing no. of strong texts <strong> texts
    ctr=-1

    while(flag<3):
        ctr+=1
        tag = a[ctr]
        if(tag.find('strong')):
            flag+=1
        elif(flag>0):
            part_list.append(tag.get_text())

    #  --------------------------------------------------------------------------------------------------------------
    # Constructing Presentation Dictionary
    presentation_dict ={}
    temp =[]
    speaker = ""
    flag = 0
    qna_head_phrases = ["question-and-answer session", "question-and-answers session", "questions-and-answers session", "questions-and-answer session"]

    while(ctr<len(a) and (" ".join(((a[ctr].get_text()).lower()).split()) not in qna_head_phrases)):
        tag = a[ctr]
        if(tag.find('strong')):
            if((tag.get_text() not in presentation_dict.keys())):
                if(flag==0):
                    speaker = tag.get_text()
                    flag=1
                elif(flag!=0):
                    presentation_dict[speaker] = temp
                    temp=[]
                    speaker = tag.get_text()
            else:
                presentation_dict[speaker] = temp
                speaker = tag.get_text()
                temp = presentation_dict[speaker]
        else:
            temp.append(tag.get_text())
        ctr+=1
    else: # To add the last speaker to presentation_dict
        presentation_dict[speaker] = temp
    
    if(ctr==len(a)):
        debug_list.append(ID)

    #  --------------------------------------------------------------------------------------------------------------
    # Construction of Questionnaire Dictionary
    ctr+=1 # To move forward from the Q-&-A Heading tag
    question_dict = {}
    temp = {}
    speaker = ""
    flag = 0
    remarks = []
    flow_ctr= 1
    while(ctr<len(a)): # Looping till end of the document
        tag =a[ctr]
        if((tag.find('strong') or (tag.get_text() in part_list))):
            if(flag==0):
                speaker = tag.get_text()
                flag=1
            else:
                temp["Speaker"] = deepcopy(speaker)
                temp["Remarks"] = remarks
                question_dict[flow_ctr] = deepcopy(temp) # Important to use deepcopy
                speaker = tag.get_text()
                remarks = []
                temp = {}
                flow_ctr+=1
        else:
            remarks.append(tag.get_text())
        ctr+=1
    else: # To add the last speaker to question_dict. 
        temp["Speaker"] = deepcopy(speaker)
        temp["Remarks"] = remarks
        question_dict[flow_ctr] = deepcopy(temp)

    #  --------------------------------------------------------------------------------------------------------------    
    # Creating the Transcript Instance 
    transcript["Date"] = date
    transcript["Presentation"] = presentation_dict
    transcript["Questionnaire"] = question_dict
    # Adding the transcript to overall nested dictionary
    ECTNestedDict["Transcript-" + str(i+1)] = deepcopy(transcript)

    #  --------------------------------------------------------------------------------------------------------------    
    # Saving the transcript Dictionary in a JSON file
    script_file_path = os.getcwd() + "/ECTNestedDict/"
    json_name = "transcript-" + str(i+1) + ".json"
    with open(script_file_path + json_name, 'w') as fp:
        json.dump(transcript, fp, sort_keys=True, indent=3)

In [ ]:
# Saving the entire transcripts in a big JSON.
script_file_path = os.getcwd() + "/"
json_name = "ECTNestedDict.json"
with open(script_file_path + json_name, 'w') as fp:
    json.dump(ECTNestedDict, fp, sort_keys=True, indent=3)

### Task 2.2
Extracting only the text files from **ECTNestedDict** & store in **ECTText**

In [ ]:
# JSON file location
file_path = os.getcwd() + "/ECTNestedDict/"
# Creating "ECTText" folder
txt_file_path = os.getcwd() + "/ECTText/"
os.mkdir(txt_file_path)
file_list = os.listdir(file_path)
debug_list = []

for i in range(1,len(file_list)+1):
    # JSON + TXT file name for Read/Write 
    json_file_name = file_path + "transcript-" + str(i) + ".json"
    txt_file_name = txt_file_path + "transcript-" + str(i) + ".txt"
    text = ""

    # Opening JSON file 
    with open(json_file_name, 'r') as transcript_file:
        transcript = json.load(transcript_file)
        presentation_dict = transcript["Presentation"]
        for presenter in presentation_dict:
            remarks = presentation_dict[presenter]
            if(remarks):
                text = text + remarks[0]
            for j in range(1, len(remarks)):
                text = text + " " + remarks[j]
        question_dict = transcript["Questionnaire"]
        for speaker in question_dict:
            remarks = question_dict[speaker]["Remarks"]
            for temp in remarks:
                text = text + " " + temp
    with open(txt_file_name,"wt") as text_file:
        text_file.write(text)


## Task 3

### Task 3.1
- Tokenization
- Lemmatization
- Stop-Words/Punctuations Removal


### Task 3.2
- Build Inverted Positional Index

#### **HINT-** Positional Index Structure

![Example Posting Index](img/1.png)

For the current example this what our Posting-List structure would look like:

```
{"hello" : { 3 : [120, 125, 278], 
             5 : [28] , 
             10 : [132, 182], 
             23 : [0, 12, 28], 
             27 : [2] 
           } }
```


In [ ]:
# import numpy as np 
import os 
import nltk 
from nltk.corpus import stopwords
from copy import deepcopy
import json
import csv

# Tokenizer
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'[a-zA-Z]+')
 
# Lemmatizer
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

nltk.download('wordnet')
nltk.download('stopwords')

In [ ]:
txt_file_path = os.getcwd() + "/ECTText/"
stop_words = set(stopwords.words('english'))
cust_stop_words = ["'s"]

file_list = os.listdir(txt_file_path)

# Adding custom words to the 
for temp in cust_stop_words:
    stop_words.add(temp)

inv_pos_index = {}

# for i in range(1,3):
for i in range(1,len(file_list)+1):
    txt_file_name = txt_file_path + "transcript-" + str(i) + ".txt"
    filtered_tokens = [] 
    with open(txt_file_name,"r") as text_file:
        text = text_file.read()
        
        # Tokenizing
        tokens = tokenizer.tokenize(text.lower())
            
        # Stop-Word Removal + Lemmatization
        for j in range(len(tokens)):
            temp = tokens[j]
            if(temp not in stop_words):
                temp1 = lemmatizer.lemmatize(temp)
                filtered_tokens.append((temp1,j)) # Storing their position in the original document as well for easier search

        #  Building Corpus
        for token in filtered_tokens:
            if(token[0] not in inv_pos_index):
                token_dict = {}
                pos_list = [token[1]]
                token_dict[i] = pos_list
                inv_pos_index[token[0]] = token_dict
            
            else:
                token_dict = inv_pos_index[token[0]]
                if(i not in token_dict.keys()):
                    pos_list = [token[1]]
                    token_dict[i] = pos_list
                    inv_pos_index[token[0]] = token_dict
                else:
                    token_dict[i].append(token[1])
        
        if(i%50==0):
            print(str(i) + " | " + str(len(inv_pos_index.keys())))

print(str(len(file_list)) + " | " + str(len(inv_pos_index.keys())))

with open("Inv_Pos_index.json", 'w') as fp:
    json.dump(inv_pos_index, fp, sort_keys=True, indent=3)

### Task 3.3
- Build & Store a Permuterm Index (In Sorted Order)

**NOTE**- Use imports from Task- (3.1 + 3.2)

In [ ]:
def GeneratePermuterm(token):
    perm_list = []
    token = token + "$"
    for i in range(len(token)):
         perm_list.append(token)
         token = token[1:] + token[0]
    
    return perm_list

def GeneratePermutermList():
    perm_list = []
    
    with open("Inv_Pos_Index.json", 'r') as fp:
        inv_pos_index = json.load(fp)
        for token in inv_pos_index:
            perm_list.extend(GeneratePermuterm(token))
        
        perm_list.sort()
        return perm_list

perm_list = GeneratePermutermList()

with open("Permuterm.txt","wt") as text_file:
    text_file.write(" ".join(perm_list))

## Task 4

Wildcard Query Retieval

### Output txt Format
Let the query be trai*

Let the two possible answers are train and trail.

Let train appreared in doc 25 in 8 and 19 position and so on.

We want the different terms to be separated by a colon ";".

So print your sample answer as:

**`"train:<25,8>,<25,19>,<29,1>,<29,6>; trail:<3,1>,<6,1>,<6,10> \n"`**

In [1]:
# Imports Required
import json
import sys

# Loading Stored Permuterms
with open("Permuterm.txt","r") as permuterm_file:
    text = permuterm_file.read()
    perm_list = text.split()

# Loading Posting List
with open("Inv_Pos_Index.json", 'r') as fp:
    inv_pos_index = json.load(fp)

In [3]:
def FindRange(token, pos):
    low_bound = pos
    high_bound = pos
    
    while((perm_list[low_bound].startswith(token)) and (low_bound > 0)):
      low_bound = low_bound - 1
    
    while((perm_list[high_bound].startswith(token)) and (high_bound < len(perm_list))):
      high_bound = high_bound + 1
    
    return(low_bound+1, high_bound-1)

def TokenSearch(token, start, end):
    if(start==end):
        if(perm_list[start].startswith(token)):
            return (start, start)
        else:
            return -1
    if(start>end or (end - start == 1)):
        return -1

    mid = int((start + end)/2)
    if(perm_list[mid].startswith(token)):
        token_range = FindRange(token,mid)
    else:
        if(token < perm_list[mid]):
            token_range = TokenSearch(token, start, mid-1)
        else:
            token_range = TokenSearch(token, mid, end)
    return token_range

def QueryToToken(query):
    # mon* CASE
    if(query[-1] == "*"):
        return query[:-1]
    # *mon CASE
    elif(query[0] == "*"):
        return (query[1:] + "$")
    # mon*mon CASE
    else:
        star_index = query.find('*')
        return(query[star_index+1:] + "$" + query[:star_index])

def TokenToWord(token):
    if(token[-1]=='$'):
        return(token[:-1])
    else:
        dollar_index = token.find("$")
        return(token[dollar_index+1:] + token[:dollar_index])

with open("query.txt","r") as query_file:
# with open(sys.argv[1],"r") as query_file:  # For taking the commandline input for the query text file
    query_list = query_file.readlines()

text_list = []    
with open("RESULTS1_18CH3FP07.txt","wt") as text_file:
        for query in query_list:
            # Generating Token for Query
            token = QueryToToken(query.rstrip("\n"))

            # Searching for approriate range for the query given
            token_range = TokenSearch(token, 0, len(perm_list)-1)

            if(token_range == -1):
                text_file.write("Query Not Found")
                text_file.write("\n")

            else:
                token_list = []
                for i in range(token_range[0],(token_range[1]+1)):
                    token_list.append(TokenToWord(perm_list[i]))

                text = ""
                for token in token_list:
                    text+= token +":"
                    token_dict = inv_pos_index[token]
                    for doc in token_dict:
                        posting_list = token_dict[doc]
                        for posting in posting_list:
                            text+= "<" + str(doc) + "," + str(posting) + ">,"
                    text = text[:-1]
                    text+= ";"
                text_file.write(text[:-1])
                text_file.write("\n")